In [100]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score , cohen_kappa_score, roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_pickle("../datos/setas/setas_balance.pkl")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [103]:
# separamos los datos en X e y

X1 = df.drop("class", axis = 1)
y1 = df["class"]

In [104]:
# separamos en train y test
x_train1, x_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size = 0.2, random_state = 42)

In [ ]:
# creamos el objeto del modelo, al igual que hacíamos en la regresión lineal
arbol = DecisionTreeClassifier(random_state =0)

# ajustamos el modelo, igual que en la regresión lienal. 
arbol.fit(x_train1, y_train1)

In [ ]:
"""

fig = plt.figure(figsize = (10,6))
tree.plot_tree(arbol, feature_names = x_train1.columns, filled = True)
plt.show()

"""

In [ ]:
# max features. Como vemos, debemos poner en nuestro modelo una profudidad máxima de 9. 

max_features = np.sqrt(len(x_train1.columns))
max_features

In [ ]:
# max depth

print(arbol.tree_.max_depth)

In [ ]:
# hacemos las predicciones sobre los dos set de datos el X_test y el X_train
y_pred_test_esta = arbol.predict(x_test1)
y_pred_train_esta = arbol.predict(x_train1)

In [ ]:
def metricas(clases_reales_test, clases_predichas_test, clases_reales_train, clases_predichas_train, modelo):
    
    # para el test
    accuracy_test = accuracy_score(clases_reales_test, clases_predichas_test)
    precision_test = precision_score(clases_reales_test, clases_predichas_test)
    recall_test = recall_score(clases_reales_test, clases_predichas_test)
    f1_test = f1_score(clases_reales_test, clases_predichas_test)
    kappa_test = cohen_kappa_score(clases_reales_test, clases_predichas_test)

    # para el train
    accuracy_train = accuracy_score(clases_reales_train, clases_predichas_train)
    precision_train = precision_score(clases_reales_train, clases_predichas_train)
    recall_train = recall_score(clases_reales_train, clases_predichas_train)
    f1_train = f1_score(clases_reales_train, clases_predichas_train)
    kappa_train = cohen_kappa_score(clases_reales_train, clases_predichas_train)
    

    
    df = pd.DataFrame({"accuracy": [accuracy_test, accuracy_train], 
                       "precision": [precision_test, precision_train],
                       "recall": [recall_test, recall_train], 
                       "f1": [f1_test, f1_train],
                       "kappa": [kappa_test, kappa_train],
                       "set": ["test", "train"]})
    
    df["modelo"] = modelo
    return df

In [ ]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results1 = metricas(y_test1, y_pred_test_esta, y_train1, y_pred_train_esta, "Decission Tree Esta I")
dt_results1

In [ ]:
# lo primero que tenemos que hacer es definir un diccionario con los hiperparámetros que queremos modificar y los valores que queremos 

param = {"max_depth": [16, 18, 20, 22, 24, 26, 28], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [4,5,6,7,8,9],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split": [50, 100, 200],
        "min_samples_leaf": [50,100, 200]} 

In [ ]:
# una vez creado el diccionario iniciaremos el modelo con GridSearch

gs = GridSearchCV(
            estimator=DecisionTreeClassifier(random_state= 42), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1) # para que no nos printee ningún mensaje en pantalla

In [ ]:
# ajustamos el modelo que acabamos de definir en el GridSearch

gs.fit(x_train1, y_train1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier# este método nos esta diciendo que el mejor modelo es aquel que tiene una profundidad de 6, que usa 4 variables predictoras para construir el modelo y que tiene  un min_samples_leaf y un min_samples_split de 10. 
mejor_modelo = gs.best_estimator_
mejor_modelo

Max_depth = 26
max_features = 9
min_samples_leaf = 50
min_samples_split = 50
random_state = 42

In [ ]:
"""

# veamos ahora que pinta tiene nuestro árbol

fig = plt.figure(figsize=(40, 20))
tree.plot_tree(mejor_modelo, feature_names=x_train1.columns, filled=True);

"""

In [ ]:
y_pred_test_esta2 = mejor_modelo.predict(x_test1)
y_pred_train_esta2 = mejor_modelo.predict(x_train1)

In [ ]:
dt_results2 = metricas(y_test1, y_pred_test_esta2, y_train1,  y_pred_train_esta2, "Decision tree Esta II")
dt_results2

In [ ]:
# vamos  a juntar los dataframes de los resultados de los modelos para poder compararlos mejor

df_decision_results = pd.concat([dt_results1, dt_results2], axis = 0)
df_decision_results

In [ ]:
# si recodáis, en la clase de métricas guardamos en un csv los resultados de las métricas del modelo
# vamos a cargar ese csv para comparar todos los modelos que hemos hecho, y comparar cuál de ellos es el mejor

df_logistic_results = pd.read_csv("../datos/setas/metricas_RLogistica.csv", index_col = 0)
df_logistic_results

In [ ]:
# concatenamos todos los resultados

df_DT_LR_results = pd.concat([df_logistic_results, df_decision_results], axis = 0).reset_index(drop=True)
df_DT_LR_results

In [ ]:
# pongamos un poco de color a nuestro dataframe para ver la comparación de los datos de una forma un poco más amigable. 
df_DT_LR_results.style.background_gradient(cmap='seismic')

In [ ]:
# ademas vamos a guardar este dataframe en un csv para 

df_DT_LR_results.to_csv("../datos/setas/metricas_modelos.csv")

In [ ]:
# vamos a crearnos un dataframe 
importancia_predictores = pd.DataFrame(
                            {'predictor': x_train1.columns,
                             'importancia': mejor_modelo.feature_importances_}
                            )


# ordenamos de mayor a menor los resultados
importancia_predictores.sort_values(by=["importancia"], ascending=False, inplace = True)

# printeamos los resultados
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores

In [ ]:
df.head(2)

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
df_predictores = importancia_predictores.reset_index(drop=True)

In [ ]:
df_predictores


In [ ]:
# lo primero que hacemos es crearnos un dataframe con los valores solo de stem_color, es decir, la primera y antepenúltima fila
stem_color= importancia_predictores.iloc[[3,34,63,67,69,71,74,78,80,81,82,83,87]]
stem_color

In [ ]:
# hacemos lo mismo para cap-surface_s
cap_surface = df_predictores.loc[[4,5,8,16,22,26,38,43,58,79,85]]
cap_surface

In [ ]:
# y para maturity
gill_attachment = df_predictores.loc[[6,17,19,23,24,25,36]]
gill_attachment

In [ ]:
# hacemos lo mismo para cap_shape
cap_shape = df_predictores.loc[[7,15,32,35,52,59,66]]
cap_shape

In [ ]:
# hacemos lo mismo para gill_color
gill_color = df_predictores.loc[[9,10,13,14,18,29,31,37,44,49,55,70]]
gill_color

In [ ]:
# hacemos lo mismo para gill_color
does_bruise_or_bleed= df_predictores.loc[[11,33]]
does_bruise_or_bleed

In [ ]:
# hacemos lo mismo para gill_color
ring_type= df_predictores.loc[[12,30,40,47,56,65,84,86]]
ring_type

In [ ]:
# hacemos lo mismo para gill_color
cap_color= df_predictores.loc[[20,28,39,46,48,51,53,60,64,68,72,73]]
cap_color

In [ ]:
# hacemos lo mismo para gill_color
habitat= df_predictores.loc[[27,42,50,57,61,75,76,77]]
habitat

In [ ]:
# hacemos lo mismo para gill_color
season= df_predictores.loc[[41,45,54,62]]
season

In [130]:

# eliminamos esas filas del dataframe donde tenemos los valores de importancia

importancia_predictores.drop(stem_color.index, inplace = True)
importancia_predictores.drop(cap_surface.index, inplace = True)
importancia_predictores.drop(gill_attachment.index, inplace = True)
importancia_predictores.drop(cap_shape.index, inplace = True)
importancia_predictores.drop(gill_color.index, inplace = True)
importancia_predictores.drop(does_bruise_or_bleed.index, inplace = True)
importancia_predictores.drop(ring_type.index, inplace = True)
importancia_predictores.drop(cap_color.index, inplace = True)
importancia_predictores.drop(habitat.index, inplace = True)
importancia_predictores.drop(season.index, inplace = True)



KeyError: '[66, 56, 67, 61, 58, 64, 55, 57, 65, 60, 62, 63, 59] not found in axis'

In [ ]:
importancia_predictores_esta

In [ ]:
stem_color, cap_surface, gill_attachment, cap_shape, gill_color, 
does_bruise_or_bleed, ring_type, cap_color, habitat, season

In [127]:
# nos creamos nuevas filas con el resultado de la suma

df_predictores.loc[11] =  ["stem_color", stem_color["importancia"].sum()]
df_predictores.loc[12] =  ["cap_surface", cap_surface["importancia"].sum()]
df_predictores.loc[13] =  ["gill_attachment", gill_attachment["importancia"].sum()]
df_predictores.loc[14] =  ["cap_shape", cap_shape["importancia"].sum()]

df_predictores.loc[15] =  ["gill_color", gill_color["importancia"].sum()]
df_predictores.loc[16] =  ["does_bruise_or_bleed", does_bruise_or_bleed["importancia"].sum()]
df_predictores.loc[17] =  ["ring_type", ring_type["importancia"].sum()]
df_predictores.loc[18] =  ["cap_color", cap_color["importancia"].sum()]

df_predictores.loc[19] =  ["habitat", habitat["importancia"].sum()]
df_predictores.loc[20] =  ["season", season["importancia"].sum()]


# ordenamos el df

df_predictores.sort_values(by = "importancia", ascending = False, inplace = True)
df_predictores

,predictor,importancia
12,cap_surface,1.649130e-01
15,gill_color,1.372445e-01
0,cap-diameter_e,1.128579e-01
13,gill_attachment,1.097970e-01
1,stem-width_e,9.707992e-02
2,stem-height_e,8.084313e-02
14,cap_shape,6.586953e-02
11,stem_color,6.335523e-02
3,stem-color_w,5.649693e-02
18,cap_color,4.819387e-02


In [ ]:
# por último ploteamos los resultados para verlo de una forma más amigable. 

plt.figure(figsize=(10,6))
sns.barplot(x = "importancia", y = "predictor", data = importancia_predictores_esta, palette="viridis");
plt.show()

## Random Forest


In [101]:
# lo primero que tenemos que hacer es definir un diccionario con los hiperparámetros que queremos modificar y los valores que queremos 
param = {"max_depth": [16, 18, 20, 22, 24, 26, 28], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [4,5,6,7,8,9],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split": [50, 100, 200],
        "min_samples_leaf": [50,100, 200]} 

In [102]:
gs_rf = GridSearchCV(
estimator=RandomForestClassifier(random_state=42), # tipo de modelo que queremos hacer
param_grid= param, # que hiperparámetros queremos que testee
cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
verbose=-1) # para que no nos printee ningún mensaje en pantalla 

In [107]:
# separamos los datos en X e y

X = df.drop("class", axis = 1)
y = df["class"]

In [108]:
# separamos en train y test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [109]:
# ajustamos el modelo que acabamos de definir en el GridSearch
# 🚨 Esta celda puede tardar en ejecutarse

gs_rf.fit(x_train, y_train) 

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [16, 18, 20, 22, 24, 26, 28],
                         'max_features': [4, 5, 6, 7, 8, 9],
                         'min_samples_leaf': [50, 100, 200],
                         'min_samples_split': [50, 100, 200]},
             verbose=-1)

In [110]:
# saquemos ahora cual es nuestro mejor bosque

bosque = gs_rf.best_estimator_
bosque 

RandomForestClassifier(max_depth=28, max_features=9, min_samples_leaf=50,
                       min_samples_split=50, random_state=42)

In [ ]:
# dibujemos los árboles de nuestro bosque
"""
for arbol in tqdm(bosque.estimators_):
fig = plt.figure(figsize=(20, 10))
tree.plot_tree(arbol, feature_names= x_train.columns, filled = True)
plt.show()

""" 

In [111]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [ ]:
# Ya tenemos definida la función de métricas, así que no vamos a repetirla

In [112]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results = metricas(y_test, y_pred_test_rf,y_train, y_pred_train_rf, "Random Forest")
dt_results 

,accuracy,precision,recall,f1,kappa,set,modelo
0,0.990912,0.991455,0.990280,0.990867,0.981824,test,Random Forest
1,0.991266,0.992204,0.990332,0.991267,0.982531,train,Random Forest


In [117]:
# igual que en la clase de decision tree, juntemos estos resultados con los obtenidos en los modelos que hemos hecho hasta ahora
# lo primero que tenemos que hacer es cargar el csv con todos los resultados que guardamos en el jupyter anterior

df_metricas = pd.read_csv("../datos/setas/metricas_modelos.csv", index_col=0)
df_metricas

,accuracy,precision,recall,f1,kappa,set,modelo
0,0.782486,0.767488,0.812794,0.789492,0.564871,test,Regresión logistica
1,0.780696,0.765383,0.808967,0.786572,0.561414,train,Regresión logistica
2,0.994217,0.993426,0.995061,0.994243,0.988434,test,Decission Tree Esta I
3,1.000000,1.000000,1.000000,1.000000,1.000000,train,Decission Tree Esta I
4,0.897321,0.875277,0.927563,0.900662,0.794594,test,Decision tree Esta II
5,0.903066,0.880480,0.932542,0.905763,0.806142,train,Decision tree Esta II


In [118]:
# concatenamos el csv anterior con los resultados obtenidos en este modelo

df_todos_resultados = pd.concat([dt_results, df_metricas], axis = 0)
df_todos_resultados 

,accuracy,precision,recall,f1,kappa,set,modelo
0,0.990912,0.991455,0.990280,0.990867,0.981824,test,Random Forest
1,0.991266,0.992204,0.990332,0.991267,0.982531,train,Random Forest
0,0.782486,0.767488,0.812794,0.789492,0.564871,test,Regresión logistica
1,0.780696,0.765383,0.808967,0.786572,0.561414,train,Regresión logistica
2,0.994217,0.993426,0.995061,0.994243,0.988434,test,Decission Tree Esta I
3,1.000000,1.000000,1.000000,1.000000,1.000000,train,Decission Tree Esta I
4,0.897321,0.875277,0.927563,0.900662,0.794594,test,Decision tree Esta II
5,0.903066,0.880480,0.932542,0.905763,0.806142,train,Decision tree Esta II


In [ ]:
df_todos_resultados.style.background_gradient(cmap='seismic') 

In [ ]:
importancia_predictores = pd.DataFrame(
{'predictor': x_train.columns,
'importancia': bosque.feature_importances_}
)


# ordenamos de mayor a menor los resultados
importancia_predictores.sort_values(by=["importancia"], ascending=False, inplace = True)

# printeamos los resultados
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores 

## volver atrás el proceso de get dummies, cuando lo terminemos en el decision tree, lo aplicaremos en este randomforest 